# Disjoint Sets(서로소 집합)
- 공통원소가 없는 두 집합을 의미한다.
- union, find 이 두개의 연산으로 서로소집합을 조작할 수 있다.
    - union : 2개의 원소가 포함된 집합을 하나의 집합으로 합치는 연산.
    - find : 특정 원소가속한 집합이 어떤 집합인지 알려주는 연산이다.
- 트리 자료구조를 이용해 집합을 표현한다.

## 서로소 집합 구현 알고리즘 - 간략
1. union (합집합) 연산을 확인하여 서로 연결된 두 노드 A, B를 확인한다.
    1. A, B의 루트 노드 A'과 B'을 찾는다.
    2. A'를 B'의 부모 노드로 설정한다. (B'가 A'을 가리키도록 한다.)
2. 모든 union (합집합) 연산을 처리할 때까지 1번과정을 반복한다.
- 작은 번호를 부모 노드로 하는 경우 많음

In [2]:
# 예시
'''
6 4
1 4
2 3
2 4
5 6
'''
def find_parent(parent,x):
    if parent[x] != x:
        # 루트노드가 아니라면 루트노드를 찾을 때까지 재귀
        parent[x] = find_parent(parent,parent[x])
    return parent[x]

def union_parent(parent,a,b):
    a = find_parent(parent,a)
    b = find_parent(parent,b)
    if a<b:
        parent[b] = a
    else:
        parent[a] = b
        # 두가지 원소를 union할 때, 작은 수를 부모로 본다.


v,e = map(int, input().split())
parent = [0] * (v+1)

for i in range(1,v+1):
    parent[i] = i

for i in range(e):
    a, b = map(int,input().split())
    union_parent(parent,a,b)

print('각 원소가 속한 집합', end = ' ')
for i in range(1,v+1):
    print(find_parent(parent,i),end = ' ')

print()

print('부모 테이블', end = ' ')
for k in range(1,v+1):
    print(parent[k],end= ' ')

각 원소가 속한 집합 1 1 1 1 5 5 
부모 테이블 1 1 1 1 5 5 

In [4]:
# 서로소 집합 알고리즘을 활용한 문제
'''
학교에서 학생들에게 0번부터 N번까지의 번호를 부여했다.
처음에는 모든 학생이 서로 다른 팀으로 구분되어 N+1개의 팀으로 존재한다.
이때 선생님은 팀합치기 연산과 같은 팀 여부확인 연산을 사용할 수 있다.
팀합치기연산 : 두 팀을 합치는 연산
팀 여부확인 : 특정 두 학생이 같은 팀에 속하는지 확인하는 연산
'''
'''
M번의 팀 합치기 연산을 수행하는데, 이 때 같은팀 여부확인 연산에 대한 연산 결과를 출력하는 프로그램 작성
'''
'''
N, M
다음 M개의 줄에 명령어가 표시된다.
0 a b : a b 팀합치기
1 a b : a b 같은팀 여부 확인
'''
def find_parent(parent, x):
    if parent[x] != x:
        # 부모가 존재한다면
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

def union_parent(parent, a, b):
    a = find_parent(parent, a) # a의 부모 누군지
    b = find_parent(parent, b) # b의 부모 누군지 
    if a<b:
        parent[b] = a # b(big)의 부모는 a(small)의 부모와 동일하다.
    else:
        parent[a] = b

N, M = map(int,input().split())
parent = [0] * (N+1)
for k in range(N+1):
    parent[k] = k
# 부모 리스트 초기화

for i in range(M):
    union_or_find, a, b = map(int,input().split())
    if union_or_find == 0:
        # 팀합치기
        union_parent(parent, a, b)
    elif union_or_find == 1:
        if find_parent(parent, a) == find_parent(parent, b):
            print('같은 팀')
        else:
            print('다른 팀')

다른 팀
다른 팀
같은 팀


# 크루스칼 알고리즘
- 최소한의 비용으로 신장 트리를 찾아야 할 경우 활용
- 신장트리 : 사이클이 발생하지 않고 서로 연결되어 있는 경우
    - N개의 도시, 도로를 놓아 도시가 서로 연결되도록 하는 최소한의 비용 구하기

## 구현
1. 간선 데이터를 비용에 따라 오름차순으로 정리한다.
2. 간선을 하나씩 확인하며 현재의 간선이 사이클을 발생시키는지 확인한다.
    1. 사이클이 발생하지 않는 경우 최소 신장 트리에 포함시킨다.
    2. 사이클이 발생하는 경우 pass
3. 모든 간선에 대하여 2번의 과정을 반복한다.

In [1]:
# 예시 - 도시 분할 계획
'''
마을엔 N개의 집과 그 집들을 연결하는 M개의 길로 이루어져 있다.
도로별로 유지비가 있다.

그 마을을 2개의 마을로 분할하려고 한다.
각 분리된 마을 내 집들이 서로 연결되도록 분할해야한다.
마을 하나엔 무조건 한 개 이상의 집이 존재한다.
'''
'''
7 12
1 2 3
1 3 2
3 2 1
2 5 2
3 4 4
7 3 6
5 1 5
1 6 2
6 4 1
6 5 3
4 5 3
6 7 4
'''
# 부모를 판별하는 함수
def find_parent(parent, num):
    if parent[num] != num:
        parent[num] = find_parent(parent, parent[num])
    return parent[num]

# a,b가 연결되도록 노드를 연결시키는 함수
def union_parent(parent,a,b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    # 작은 번호의 노드를 부모노드로 삼는다.
    if a<b:
        parent[b] = a
    else:
        parent[a] = b

house_num, road_num = map(int,input().split())
town_info = []
for _ in range(road_num):
    a,b,cost = map(int,input().split())
    town_info += [(cost,a,b)]
town_info.sort()
# 집의 개수, 도로의 개수를 입력받고
# (연결된 도로 별 유지비용, a,b)가 입력된 리스트들을 꾸린 후 sort

total_price = 0
maximum_price = 0
# 신장 리스트를 꾸리면서 추가하며 총 금액을 확인할 total_price
# 마지막에 추가한 도로의 금액을 maximum_price로 보며 알고리즘 진행

parent = [0] * (house_num+1)
for k in range(1,house_num+1):
    parent[k] = k
# 부모 노드 초기화
    
for info in town_info:
    # town_info데이터를 확인한다. sort했으므로 사이클 여부를 판별하고 추가한다.
    cost, a, b = info
    if find_parent(parent,a) != find_parent(parent,b):
        union_parent(parent,a,b)
        total_price += cost
        maximum_price = cost

print('만약 마을이 하나라면 총 유지금액은', total_price)
print('거기서 가장 비용이 큰 도로 유지비용은', maximum_price)
print('따라서 결론은 총 금액 - 가장 비싼 유지비용', total_price-maximum_price)

만약 마을이 하나라면 총 유지금액은 12
거기서 가장 비용이 큰 도로 유지비용은 4
따라서 결론은 총 금액 - 가장 비싼 유지비용 8


# 위상정렬 (Topology sort)
- 방향 그래프의 모든 노드를 '방향성에 거스르지 않도록 순서대로 나열하는 것'
- 큐 활용
- 진입차수 리스트를 활용한다(몇개가 그 노드를 향하냐)
- 커리큘럼 문제 등에 활용

In [5]:
# 커리큘럼
'''
N개의 강의 (1번부터 N번까지)
동시에 여러 강의를 들을 수 있다고 가정한다.
'''
'''
5
10 -1
10 1 -1
4 1 -1
4 3 1 -1
3 3 -1
'''
N = int(input())

time_list = [0] * (N+1)
# index에 해당하는 수업을 듣는데 소요되는 시간리스트
result = [0] * (N+1)
# 결과에 사용할 시간리스트
class_list = [[] for _ in range(N+1)]
# index에 해당하는 수업이 어떤 수업의 선수수업인지 기록하기 위한 리스트
indegree = [0] * (N+1)
# 해당 수업이 몇개의 수업을 들어야 들을 수 있는 수업인지 기록하기 위한 리스트

for _ in range(1,N+1):
    info = list(map(int,input().split()))
    time_list[_] += info[0]
    result[_] += info[0] # copy.deepcopy(time) 대체
    for x in info[1:-1]:
        class_list[x] += [_]
        indegree[_] += 1

Q = []
for k in range(1,len(indegree)):
    if indegree[k] == 0:
        Q.append(k)
        # indegree : [0, 0, 1, 1, 2, 1]

while Q:
    current = Q.pop(0)
    for i in class_list[current]:
        '''
        class_list : [[], [2, 3, 4], [], [4, 5], [], []]
        time_list : [0, 10, 10, 4, 4, 3]
        '''
        result[i] = result[current] + time_list[i]
        # 문제 예시처럼 max를 써야하는 이유가 무엇? 스터디에서 물어보기
        '''
        현 위치의 class_list를 확인했을 때, 
        해당 과목을 필요로 하는 과목의 번호 result를 수정한다.
        그 값은 current과목 시간 + 해당 과목의 시간으로 정의할 수 있다.
        '''
        '''
        순서
        CURRENT = 1
        - 2를 i로 가져온 경우
        result[2] = result[1] + time_list[2]
        result = [0,10,20,4,4,3]
        indegree = [0,0,1,1,2,1] -> [0,0,0,1,2,1]
        Q = [2]
        - 3을 i로 가져온 경우
        result[3] = result[1] + time_list[3]
        result = [0,10,20,14,4,3]
        indegree = [0,0,0,1,2,1] -> [0,0,0,0,2,1]
        Q = [2,3]
        - 4를 i로 가져온 경우
        result[4] = result[1] + time_list[4]
        result = [0,10,20,14,14,3]
        indegree = [0,0,0,0,2,1] -> [0,0,0,0,1,1]
        Q = [2,3]

        CURRENT = 2
        pass
        Q = [3]

        CURRENT = 3
        Q = []
        - 4를 i로 가져온 경우
        result[4] = result[3] + time_list[4]
        result = [0,10,20,14,18,3]
        indegree = [0,0,0,0,2,1] -> [0,0,0,0,0,1]
        Q = [4]
        - 5를 i로 가져온 경우
        result[5] = result[3] + time_list[5]
        result = [0,10,20,14,18,17]
        indegree = [0,0,0,0,2,1] -> [0,0,0,0,0,0]

        CURRENT = 4
        Q = []
        while break
        '''
        indegree[i] -= 1
        if indegree[i] ==0:
            Q.append(i)

for k in range(1,N+1):
    print(result[k])

10
20
14
18
17
